In [1]:

import pyspark
import requests
import json
from pyspark.sql import SparkSession, functions as F



In [2]:
spark = SparkSession.builder.appName("Kafka Python Task User ").getOrCreate()

22/11/22 08:03:02 WARN Utils: Your hostname, Code resolves to a loopback address: 127.0.1.1; using 192.168.18.113 instead (on interface enp3s0)
22/11/22 08:03:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 08:03:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
users_df = spark.read.format('json').option('multiLine', 'true').load('response_consumer1.json')
users_df.printSchema()

root
 |-- contributions_count: long (nullable = true)
 |-- github_profile: string (nullable = true)
 |-- gravatar_id: string (nullable = true)
 |-- id: long (nullable = true)
 |-- link: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- organisations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- avatar_url: string (nullable = true)
 |    |    |-- link: string (nullable = true)
 |    |    |-- login: string (nullable = true)
 |-- pull_requests: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- issue_url: string (nullable = true)
 |    |    |-- repo_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- twitter_profile: string (nullable = true)



In [4]:
users_df.count()

61

In [5]:
#Transformation1: Making the  organization dataframe for organization column(nested)
users_df=users_df.withColumnRenamed("gravatar_id","gravater")\
                    .withColumnRenamed("github_profile","github")\
                    .withColumnRenamed("twitter_profile","twitter")

In [6]:
users_df.columns

['contributions_count',
 'github',
 'gravater',
 'id',
 'link',
 'nickname',
 'organisations',
 'pull_requests',
 'twitter']

In [7]:
users_df.printSchema()

root
 |-- contributions_count: long (nullable = true)
 |-- github: string (nullable = true)
 |-- gravater: string (nullable = true)
 |-- id: long (nullable = true)
 |-- link: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- organisations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- avatar_url: string (nullable = true)
 |    |    |-- link: string (nullable = true)
 |    |    |-- login: string (nullable = true)
 |-- pull_requests: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- issue_url: string (nullable = true)
 |    |    |-- repo_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |-- twitter: string (nullable = true)



In [8]:
user=users_df.select(["id","nickname","gravater","github","twitter","link"])
user.printSchema()

root
 |-- id: long (nullable = true)
 |-- nickname: string (nullable = true)
 |-- gravater: string (nullable = true)
 |-- github: string (nullable = true)
 |-- twitter: string (nullable = true)
 |-- link: string (nullable = true)



In [9]:

user.show()

+-----+-------------+--------+--------------------+--------------------+--------------------+
|   id|     nickname|gravater|              github|             twitter|                link|
+-----+-------------+--------+--------------------+--------------------+--------------------+
|20344|  bastelfreak|        |https://github.co...|                null|https://24pullreq...|
| 9708|   adamchainz|        |https://github.co...|                null|https://24pullreq...|
|27452|   naomi-lgbt|        |https://github.co...|                null|https://24pullreq...|
|  773|        Trott|        |https://github.co...|                null|https://24pullreq...|
| 6031|   localheinz|        |https://github.co...|https://twitter.c...|https://24pullreq...|
| 3914|     nschonni|        |https://github.co...|                null|https://24pullreq...|
|27109|    snitin315|        |https://github.co...|                null|https://24pullreq...|
|10386|  changeworld|        |https://github.co...|https://t

In [18]:
from kafka import KafkaProducer
from json import dumps
# creating the kafka producer 
producer_user= KafkaProducer(bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x:dumps(x).encode('utf-8'))
count=user.select("id").count()
# collect data from the  dataframe and to topic
for i in range(count-1):
    producer_user.send("user",user.collect()[i])

In [10]:
organisation=users_df.select("organisations")
organisation.show()


+--------------------+
|       organisations|
+--------------------+
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|                  []|
|                  []|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|[{https://avatars...|
|                  []|
|[{https://avatars...|
|                  []|
+--------------------+
only showing top 20 rows



In [12]:
organisation_df = users_df.select(\
    F.col("organisations.login"),\
    F.col("organisations.avatar_url"),\
    F.col("organisations.link"))

organisation_df.printSchema()
    

root
 |-- login: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- avatar_url: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- link: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
organisation_df.na.drop("all")
organisation_df.show()

+--------------------+--------------------+--------------------+
|               login|          avatar_url|                link|
+--------------------+--------------------+--------------------+
|[example42, secpu...|[https://avatars....|[https://24pullre...|
|[django, djangolo...|[https://avatars....|[https://24pullre...|
|[freeCodeCamp, st...|[https://avatars....|[https://24pullre...|
|[ucsf-ckm, ilios,...|[https://avatars....|[https://24pullre...|
|[zendframework, b...|[https://avatars....|[https://24pullre...|
|[flot, wet-boew, ...|[https://avatars....|[https://24pullre...|
|[collective, webp...|[https://avatars....|[https://24pullre...|
|                  []|                  []|                  []|
|                  []|                  []|                  []|
|[geoalchemy, XCSo...|[https://avatars....|[https://24pullre...|
|       [CrowdStrike]|[https://avatars....|[https://24pullre...|
|[apache, openshif...|[https://avatars....|[https://24pullre...|
|      [plasma-umass]|[ht

In [19]:
from kafka import KafkaProducer
from json import dumps
# creating the kafka producer 
producer_org = KafkaProducer(bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x:dumps(x).encode('utf-8'))

count=organisation_df.select("login").count()
# collect data from the  dataframe and to topic
for i in range(count-1):
    producer_org.send("organisation",organisation_df.collect()[i])

In [14]:
pull_request=users_df.select("pull_requests")
pull_request.show()

+--------------------+
|       pull_requests|
+--------------------+
|[{, 2018-12-01 20...|
|[{null, 2016-12-0...|
|[{# Pull Request\...|
|[{, 2015-12-08 04...|
|[{This PR\r\n\r\n...|
|[{Travis seemed t...|
|[{Update CLA link...|
|[{, 2016-12-11 16...|
|[{## Purpose\r\n$...|
|[{The `ProjectStr...|
|[{, 2018-12-23 17...|
|[{This adds a sim...|
|[{We recently swi...|
|[{Added Tutorial ...|
|[{It's not quite ...|
|[{, 2015-12-08 15...|
|[{There is a slig...|
|[{null, 2021-12-1...|
|[{Reuse variables...|
|[{Changed docker....|
+--------------------+
only showing top 20 rows



In [15]:
pull_request_df = users_df.select(\
    F.col("pull_requests.title"),\
    F.col("pull_requests.issue_url"),\
    F.col("pull_requests.repo_name"),\
    F.col("pull_requests.body"),\
    F.col("pull_requests.created_at"),
    )

pull_request_df.printSchema()

root
 |-- title: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- issue_url: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- repo_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- body: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- created_at: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [16]:
pull_request_df.na.drop("all")
pull_request_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|           issue_url|           repo_name|                body|          created_at|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Update multiple ...|[https://github.c...|[bastelfreak/pupp...|[, null, Both pac...|[2018-12-01 20:14...|
|[Upgrade requirem...|[https://github.c...|[adamchainz/pytes...|[null, null, null...|[2016-12-06 08:48...|
|[feat: initial in...|[https://github.c...|[nhcarrigan/adven...|[# Pull Request\r...|[2021-12-01 15:44...|
|[doc: copyedit ch...|[https://github.c...|[nodejs/node, nod...|[, , , , , , , `e...|[2015-12-08 04:53...|
|[Fix: Test name, ...|[https://github.c...|[thephpleague/rou...|[This PR\r\n\r\n*...|[2015-12-14 10:49...|
|[Bump the ruby ve...|[https://github.c...|[github/governmen...|[Travis seemed to...|[2016-12-01 17:00...|
|[docs: update CLA...|[https://github

In [33]:

from kafka import KafkaProducer
from json import dumps
# creating the kafka producer 
producer_pull = KafkaProducer(bootstrap_servers=['localhost:9092'],
    value_serializer=lambda x:dumps(x).encode('utf-8'))

count=pull_request_df.select("title").count()
# collect data from the  dataframe and to topic
for i in range(count-1):
    producer_pull.send("pull_request",pull_request_df.collect()[i])